In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math


In [4]:
# tf.__version__
# First convolutional Layer
filter_size1 = 5 #5px X 5px
num_filters1 = 16 # A total of 16 filters

# Conv2
filter_size2 = 5
num_filters2 = 36

# Fully-connected Layer
fc_size = 128

# Downloading Data
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [5]:
data.test.cls = np.argmax(data.test.labels, axis=1)

In [ ]:
img_size = 28
img_size_flat = img_size * img_size
# Used to reshape arrays
img_shape = (img_size, img_size)
# Color chanels for the images
num_channels = 1
# Classes for digits
num_classes = 10

# Plotting images in a grid and writing the true & predicted classes below
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    fig, axes = plt.subplot(3,3)
    